In [2]:
import pygazeanalyser

In [17]:
# analysis script for natural viewing experiment
#
# version 1 (1 Mar 2014)

__author__ = "Edwin Dalmaijer"

# native
import os

# custom
from pygazeanalyser.edfreader import read_edf, happy
from pygazeanalyser.gazeplotter import draw_fixations, draw_heatmap, draw_scanpath, draw_raw

# external
import numpy as np

ImportError: cannot import name 'happy' from 'pygazeanalyser.edfreader' (/Users/isaacchristian/Desktop/Princeton/RESEARCH/CANS/pygaze_stuff/examples/analysis/pygazeanalyser/edfreader.py)

In [ ]:
for i in np.arange(n):
        sigbufs[i, :] = f.readSignal(i)

In [ ]:
import pyedflib

file_name = pyedflib.data.get_generator_filename()
f = pyedflib.EdfReader(file_name)
n = f.signals_in_file
signal_labels = f.getSignalLabels()
sigbufs = np.zeros((n, f.getNSamples()[0]))


In [3]:
import mne

In [20]:
def read_edf_basic(filename, start, stop=None, missing=0.0, debug=False):
    """Returns a list with dicts for every trial. A trial dict contains the
    following keys:
        x		-	numpy array of x positions
        y		-	numpy array of y positions
        size		-	numpy array of pupil size
        time		-	numpy array of timestamps, t=0 at trialstart
        trackertime	-	numpy array of timestamps, according to EDF
        events	-	dict with the following keys:
                        Sfix	-	list of lists, each containing [starttime]
                        Ssac	-	list of lists, each containing [starttime]
                        Sblk	-	list of lists, each containing [starttime]
                        Efix	-	list of lists, each containing [starttime, endtime, duration, endx, endy]
                        Esac	-	list of lists, each containing [starttime, endtime, duration, startx, starty, endx, endy]
                        Eblk	-	list of lists, each containing [starttime, endtime, duration]
                        msg	-	list of lists, each containing [time, message]
                        NOTE: timing is in EDF time!

    arguments
    filename		-	path to the file that has to be read
    start		-	trial start string

    keyword arguments
    stop			-	trial ending string (default = None)
    missing		-	value to be used for missing data (default = 0.0)
    debug		-	Boolean indicating if DEBUG mode should be on or off;
                if DEBUG mode is on, information on what the script
                currently is doing will be printed to the console
                (default = False)

    returns
    data			-	a list with a dict for every trial (see above)
    """

    # # # # #
    # debug mode

    if debug:
        def message(msg):
            print(msg)
    else:
        def message(msg):
            pass


    # # # # #
    # file handling

    # check if the file exists
    if os.path.isfile(filename):
        # open file
        message("opening file '%s'" % filename)
        f = open(filename, 'r')
    # raise exception if the file does not exist
    else:
        raise Exception("Error in read_edf: file '%s' does not exist" % filename)

    # read file contents
    message("reading file '%s'" % filename)
    raw = f.readlines()

    # close file
    message("closing file '%s'" % filename)
    f.close()
    return raw

In [45]:
# # # # #
# CONSTANTS

# PARTICIPANTS
#PPS = ['demo','demo2']

# DIRECTORIES
# paths
DIR = os.getcwd()
IMGDIR = os.path.join(DIR, 'imgs')
DATADIR = os.path.join(DIR, 'data')
PLOTDIR = os.path.join(DIR, 'plots')
OUTPUTFILENAME = os.path.join(DIR, "output.txt")
# check if the image directory exists
if not os.path.isdir(IMGDIR):
	raise Exception("ERROR: no image directory found; path '%s' does not exist!" % IMGDIR)
# check if the data directory exists
if not os.path.isdir(DATADIR):
	raise Exception("ERROR: no data directory found; path '%s' does not exist!" % DATADIR)
# check if output directorie exist; if not, create it
if not os.path.isdir(PLOTDIR):
	os.mkdir(PLOTDIR)

# DATA FILES
SEP = '\t' # value separator
EDFSTART = "!MODE RECORD" # EDF file trial start message
EDFSTOP = "!V TRIAL_VAR" # EDF file trial end message
TRIALORDER = [EDFSTART, 'image online','image offline', EDFSTOP]
INVALCODE = 0.0 # value coding invalid data

# EXPERIMENT SPECS
DISPSIZE = (1024,768) # (px,px)
SCREENSIZE = (39.9,29.9) # (cm,cm)
SCREENDIST = 61.0 # cm
PXPERCM = np.mean([DISPSIZE[0]/SCREENSIZE[0],DISPSIZE[1]/SCREENSIZE[1]]) # px/cm

In [46]:
DATADIR

'/Users/isaacchristian/Desktop/Princeton/RESEARCH/CANS/pygaze_stuff/examples/analysis/data'

In [47]:
PPS = ['sub007_2023_04_06_09_38']
PPS = ['sub008_2023_04_11_11_19']

In [53]:

# loop through all participants
for ppname in PPS:

    # GAZE DATA
    print("loading gaze data")

    # path
    fp = os.path.join(DATADIR, '%s.asc' % ppname)

    # check if the path exist
    if not os.path.isfile(fp):
        # if not, check if the EDF exists
        edfp = os.path.join(DATADIR, '%s.edf' % ppname)
        if os.path.isfile(edfp):
            # if it does, convert if usinf edf2asc.exe
            os.system("edf2asc %s" % edfp)
            # load ASCII
            fp = os.path.join(DATADIR, '%s.asc' % ppname)
        # if it does not exist, raise an exception
        else:
            raise Exception("No eye data file (neither ASC, nor EDF) file found for participant '%s' (tried paths:\nASC: %s\nEDF: %s" % (ppname, fp, edfp))

    # read the file
    # edfdata[trialnr]['time'] = list of timestamps in trialnr
    # edfdata[trialnr]['size'] = list of pupil size samples in trialnr
    #edfdata = read_edf_basic(fp, EDFSTART, stop=EDFSTOP, missing=INVALCODE, debug=False)
    edfdata = read_edf(fp, EDFSTART, stop=EDFSTOP, missing=INVALCODE, debug=False)

    # NEW OUTPUT DIRECTORIES
    # create a new output directory for the current participant
    pplotdir = os.path.join(PLOTDIR, ppname)
    # check if the directory already exists
    if not os.path.isdir(pplotdir):
        # create it if it doesn't yet exist
        os.mkdir(pplotdir)




loading gaze data


In [32]:
len(edfdata

1998764

In [54]:
edfdata

[{'x': array([1285.4, 1285.2, 1285.1, ...,  962.2,  962.3,  962.3]),
  'y': array([677.6, 677.4, 678.2, ..., 699.9, 698.9, 698.8]),
  'size': array([3761., 3759., 3758., ..., 4456., 4453., 4451.]),
  'time': array([   0,    1,    2, ..., 8823, 8824, 8825]),
  'trackertime': array([34771, 34772, 34773, ..., 43594, 43595, 43596]),
  'events': {'Sfix': [34778,
    35228,
    35385,
    35808,
    36069,
    36850,
    37394,
    37956,
    38063,
    38364,
    38528,
    39048,
    39202,
    39678,
    39855,
    40370,
    40506,
    40757,
    40919,
    41450,
    41527,
    42154,
    42921,
    43252,
    43469],
   'Ssac': [35178,
    35371,
    35786,
    36054,
    36832,
    37376,
    37795,
    38025,
    38340,
    38504,
    38900,
    39179,
    39641,
    39819,
    40188,
    40460,
    40733,
    40895,
    41282,
    41508,
    42133,
    42745,
    43202,
    43441],
   'Sblk': [37817, 38926, 40209, 41315, 42786],
   'Efix': [[34778, 35177, 400, 1283.5, 684.6],
    [3

In [ ]:
# find messages # 

In [42]:
arr = [word for idx, word in enumerate(edfdata[50:]) if word[:3] == 'MSG' ]

In [44]:
arr[:100]

['MSG\t34680 DRIFTCORRECT R RIGHT at 1280,720  OFFSET 0.85 deg.  7.9,-35.3 pix.\n',
 'MSG\t34770 RECCFG CR 1000 2 0 R\n',
 'MSG\t34770 ELCLCFG BTABLER\n',
 'MSG\t34770 GAZE_COORDS 0.00 0.00 2559.00 1439.00\n',
 'MSG\t34770 THRESHOLDS R 131 240\n',
 'MSG\t34770 ELCL_WINDOW_SIZES 176 188 0 0\n',
 'MSG\t34770 CAMERA_LENS_FOCAL_LENGTH 38.00 \n',
 'MSG\t34770 PUPIL_DATA_TYPE RAW_AUTOSLIP\n',
 'MSG\t34770 ELCL_PROC ELLIPSE  (5)\n',
 'MSG\t34770 ELCL_EFIT_PARAMS 1.01 4.00  0.15 0.05  0.65 0.65  0.00 0.00 0.30\n',
 'MSG\t34771 !MODE RECORD CR 1000 2 0 R\n',
 'MSG\t43609 !V TRIAL_VAR condition 000000029020\n',
 'MSG\t44268 RECCFG CR 1000 2 0 R\n',
 'MSG\t44268 ELCLCFG BTABLER\n',
 'MSG\t44268 GAZE_COORDS 0.00 0.00 2559.00 1439.00\n',
 'MSG\t44268 THRESHOLDS R 131 240\n',
 'MSG\t44268 ELCL_WINDOW_SIZES 176 188 0 0\n',
 'MSG\t44268 CAMERA_LENS_FOCAL_LENGTH 38.00 \n',
 'MSG\t44268 PUPIL_DATA_TYPE RAW_AUTOSLIP\n',
 'MSG\t44268 ELCL_PROC ELLIPSE  (5)\n',
 'MSG\t44268 ELCL_EFIT_PARAMS 1.01 4.00  0.15

In [33]:
edfdata

['** CONVERTED FROM /Users/isaacchristian/Desktop/Princeton/RESEARCH/CANS/pygaze_stuff/examples/analysis/data/sub008_2023_04_11_11_19.edf using edfapi 4.2.1.0 MacOS X   standalone Jun 18 2021 on Thu Apr 20 09:37:27 2023\n',
 '** DATE: Tue Apr 11 03:35:11 2023\n',
 '** TYPE: EDF_FILE BINARY EVENT SAMPLE TAGGED\n',
 '** VERSION: EYELINK II 1\n',
 '** SOURCE: EYELINK CL\n',
 '** EYELINK II CL v5.15 Jan 24 2018\n',
 '** CAMERA: Eyelink GL Version 1.2 Sensor=AJ7\n',
 '** SERIAL NUMBER: CLG-BEA12\n',
 '** CAMERA_CONFIG: BEA12200.SCD\n',
 '** RECORDED BY cans_4-24-23_EYE-pix.py\n',
 '**\n',
 '\n',
 'MSG\t0 DISPLAY_COORDS  0 0 2559 1439\n',
 'INPUT\t9215\t0\n',
 'MSG\t15950 !CAL \n',
 '>>>>>>> CALIBRATION (HV5,P-CR) FOR RIGHT: <<<<<<<<<\n',
 'MSG\t15950 !CAL Calibration points:  \n',
 'MSG\t15950 !CAL -65.2, -61.6        -0,    160   \n',
 'MSG\t15950 !CAL -66.3, -79.6        -0,  -2604   \n',
 'MSG\t15950 !CAL -64.1, -45.1        -0,   2862   \n',
 'MSG\t15950 !CAL -94.7, -62.0     -3768,    

In [ ]:
# # # # #
# PLOTS

print("plotting gaze data")

# loop through trials
for trialnr in range(len(data)):

    # load image name, saccades, and fixations
    imgname = data[trialnr][header.index("image")]
    saccades = edfdata[trialnr]['events']['Esac'] # [starttime, endtime, duration, startx, starty, endx, endy]
    fixations = edfdata[trialnr]['events']['Efix'] # [starttime, endtime, duration, endx, endy]

    # paths
    imagefile = os.path.join(IMGDIR,imgname)
    rawplotfile = os.path.join(pplotdir, "raw_data_%s_%d" % (ppname,trialnr))
    scatterfile = os.path.join(pplotdir, "fixations_%s_%d" % (ppname,trialnr))
    scanpathfile =  os.path.join(pplotdir, "scanpath_%s_%d" % (ppname,trialnr))
    heatmapfile = os.path.join(pplotdir, "heatmap_%s_%d" % (ppname,trialnr))

    # raw data points
    draw_raw(edfdata[trialnr]['x'], edfdata[trialnr]['y'], DISPSIZE, imagefile=imagefile, savefilename=rawplotfile)

    # fixations
    draw_fixations(fixations, DISPSIZE, imagefile=imagefile, durationsize=True, durationcolour=False, alpha=0.5, savefilename=scatterfile)

    # scanpath
    draw_scanpath(fixations, saccades, DISPSIZE, imagefile=imagefile, alpha=0.5, savefilename=scanpathfile)

    # heatmap		
    draw_heatmap(fixations, DISPSIZE, imagefile=imagefile, durationweight=True, alpha=0.5, savefilename=heatmapfile)
